<a href="https://colab.research.google.com/github/Dania0322/Project_Dania/blob/main/NewProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries, Mujoco, Spinning up , video saving.


In [ ]:
#@title ..
!apt-get install -y \
   libgl1-mesa-dev \
   libgl1-mesa-glx \
   libglew-dev \
   libosmesa6-dev \
   software-properties-common

!apt-get install -y patchelf 

!pip install free-mujoco-py

In [ ]:
#@title ..

!git clone https://github.com/openai/spinningup.git
%cd spinningup
!pip install -e .
from google.colab import drive
drive.mount('/content/drive')
# Libraries for visualization. Code taken from CIS732 KSU
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay scikit-video > /dev/null 2>&1

%tensorflow_version 1.x
import tensorflow as tf
tf.enable_eager_execution()
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
import time


In [ ]:
import mujoco_py
import gym
env = gym.make("InvertedPendulum-v2")
env.reset()

**Train the algorithm PPO and DDPG on Inverted Pendulum environment**

*(This step was runned previousl to save the trained models, it takes too long, please skip it and run the next cell!!!)* In this step we are going to train both algorithmms under different **activation functions**, **hidden layer size**, **number of epochs** and **episodes per epoch**. After training, every model is tested in 10 episodes of 1000 steps each one with 10 different random seeds. After that, the Average Total Reward is computed and saved for further analysis.

In [ ]:
#@title .
####### THIS STEP CAN TAKEN TOO LONG, files are already saved in "Project" folder, continue with the next cells please
from pandas.core.frame import DataFrame
from spinup import ppo_pytorch as ppo
from spinup import ddpg_pytorch as ddpg
import torch
import gym
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

def testEnvironment(output_dir):
  #This function runs a test over the trained model, 10 episodes, 1000 steps per episode
 modelPath = output_dir +'/pyt_save/model.pt'
 ac = torch.load(modelPath)  
 AverageReward = []
 for i in range(10):
    rewards =0
    env = gym.make("InvertedPendulum-v2")
    env.seed(i)
    obs = env.reset() 
    done = False
    while not done: 
      action = ac.act(torch.as_tensor(obs, dtype=torch.float32))
      obs, reward, done, info = env.step(action)
      rewards = rewards +reward
    AverageReward.append(rewards)
 avgReward =  sum(AverageReward)/10
 return avgReward, AverageReward
def runAlgos(hidden_sizes,activation,algo,i,steps_per_epoch,epochs):
    # This function trained a model with a specific configuration
     ac_kwargs = dict(hidden_sizes=hidden_sizes, activation=activation)
     output_dir='/content/drive/MyDrive/Project/experiments2/'+algo+'-'+str(i)
     logger_kwargs = dict(output_dir=output_dir, exp_name='experiment')
     if algo=='DDPG':
        ddpg(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=steps_per_epoch, epochs=epochs, logger_kwargs=logger_kwargs)    
     if algo=='PPO': 
        ppo(env_fn=env_fn,ac_kwargs=ac_kwargs ,steps_per_epoch=steps_per_epoch, epochs=epochs, logger_kwargs=logger_kwargs)
     avgReward = testEnvironment(output_dir)     
     return avgReward

env_fn = lambda : gym.make('InvertedPendulum-v2')
actFun =['ReLu','Tanh']
actFuncs = {0:torch.nn.ReLU,1:torch.nn.Tanh}
hiddenSizes = {0:[64,64],1:[100,50,25]}
epochs1 = [10, 20, 50, 100]
steps_per_epoch1 = [1500,1500,2000,2000]
algos =['DDPG','PPO']
Table1 = []
i = 0
for h in range(len(algos)):
 for x in range(len(actFuncs)):
  for y in range(len(hiddenSizes)):
    for z in range(len(epochs1)):
         avgReward, AverageReward = runAlgos(hiddenSizes[y],actFuncs[x],algos[h],i,steps_per_epoch1[z],epochs1[z])
         intervals = st.t.interval(alpha=0.95, df=len(AverageReward)-1, loc=np.mean(AverageReward), scale=st.sem(AverageReward))
         st_dev = np.std(AverageReward)
         Table1.append((steps_per_epoch1[z], epochs1[z],hiddenSizes[y], actFun[x], algos[h], avgReward,np.squeeze(AverageReward),np.squeeze(intervals),np.squeeze(st_dev) ))
         df = DataFrame(Table1,columns =['Steps_Epoch',  'Epochs','Hidden_Layers', 'Act_Function','Algo','Avg_Reward','Rewards','Intervals','Stand_dev'])
         df.to_csv("/content/drive/MyDrive/Table3.csv",sep='\t')  
         i=i+1

After training the models, let us check the Average return with the next table


In [5]:
import pandas as pd
results = pd.read_csv('/content/Tables/Table3.csv',sep='\t')
table1 = pd.pivot_table(results,values=["Avg_Reward","Stand_dev"],index=["Epochs","Steps_Epoch"],columns=["Algo","Act_Function","Hidden_Layers"])
print('Table 1. Average of Total cumulative Reward, standard deviation and confidence intervals, testing on 10 episodes (1000 steps per episode) with 10 different random seeds')
table1

Table 1. Average of Total cumulative Reward, standard deviation and confidence intervals, testing on 10 episodes (1000 steps per episode) with 10 different random seeds


Avg_Reward                                  \
Algo                        DDPG                                   
Act_Function                ReLu                   Tanh            
Hidden_Layers      [100, 50, 25] [64, 64] [100, 50, 25] [64, 64]   
Epochs Steps_Epoch                                                 
10     1500                 36.0     43.0          38.7     43.7   
20     1500               1000.0   1000.0         788.6   1000.0   
50     2000                 56.7     59.4          56.5   1000.0   
100    2000               1000.0     39.3        1000.0   1000.0   

                                                                  \
Algo                         PPO                                   
Act_Function                ReLu                   Tanh            
Hidden_Layers      [100, 50, 25] [64, 64] [100, 50, 25] [64, 64]   
Epochs Steps_Epoch                                                 
10     1500                151.7     72.9          81.2     90.9   
20     1500                610.7   1000.0         664.1   1000.0   
50     2000               1000.0   1000.0        1000.0   1000.0   
100    2000               1000.0   1000.0        1000.0   1000.0   

                       Stand_dev                                    \
Algo                        DDPG                                     
Act_Function                ReLu                    Tanh             
Hidden_Layers      [100, 50, 25]  [64, 64] [100, 50, 25]  [64, 64]   
Epochs Steps_Epoch                                                   
10     1500             0.894427  0.774597      0.458258  1.552417   
20     1500             0.000000  0.000000    284.998667  0.000000   
50     2000             0.458258  2.416609      3.413210  0.000000   
100    2000             0.000000  0.458258      0.000000  0.000000   

                                                                      
Algo                         PPO                                      
Act_Function                ReLu                     Tanh             
Hidden_Layers      [100, 50, 25]   [64, 64] [100, 50, 25]   [64, 64]  
Epochs Steps_Epoch                                                    
10     1500            82.136533  34.815083     29.440788  35.061232  
20     1500           296.548158   0.000000    315.493724   0.000000  
50     2000             0.000000   0.000000      0.000000   0.000000  
100    2000             0.000000   0.000000      0.000000   0.000000

The results suggest that both algorithms start to get the maximum Average Return (1000) with 30000 total environment interactions (**20 epochs**, **1500 steps per epoch**). The **sample efficiency** (number of samples needed to train a good policy) is a pretty important aspect to consider in deep RL algorithms. This is because in real-life implementation getting samples is **slower** and **expensive**. As 30000 samples provide good policies, this is the configuration that we are going to explore and use for the next evaluations. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
results = pd.read_csv('/content/drive/MyDrive/Project/Table3.csv',sep='\t')
Times =results.loc[[1,5,9,13,17,21,25,29]]
times = []
for i in np.linspace(1, 13, num=4,dtype=int):
   progress = pd.read_csv('/content/drive/MyDrive/Project/experiments2/DDPG-'+str(i)+'/progress.txt',sep='\t')
   time = progress[['Time']].to_numpy()
   times.append(np.squeeze(time[19]))
for i in np.linspace(17, 29, num=4,dtype=int):
   progress = pd.read_csv('/content/drive/MyDrive/Project/experiments2/PPO-'+str(i)+'/progress.txt',sep='\t')
   time = progress[['Time']].to_numpy()
   times.append(np.squeeze(time[19]))
Times['Times']=times
print('Table 2. Total training time of different configurations for 30000 total environment interactions (20 epochs, 1500 steps per epoch)')
table2 = pd.pivot_table(Times,values="Times",index=["Act_Function","Hidden_Layers"],columns=["Algo"])
table2


Table 2. Total training time of different configurations for 30000 total environment interactions (20 epochs, 1500 steps per epoch)


Algo                              DDPG        PPO
Act_Function Hidden_Layers                       
ReLu         [100, 50, 25]  191.893610  31.879787
             [64, 64]       150.726783  28.395020
Tanh         [100, 50, 25]  207.827065  35.716079
             [64, 64]       160.529953  29.650362

We can also check the training times with a total number steps of 200000 and we can also see that in general PPO is faster

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
results = pd.read_csv('/content/drive/MyDrive/Project/Table3.csv',sep='\t')
Times2 =results.loc[[3,7,11,15,19,23,27,31]]
times = []
for i in np.linspace(3, 15, num=4,dtype=int):
   progress = pd.read_csv('/content/drive/MyDrive/Project/experiments2/DDPG-'+str(i)+'/progress.txt',sep='\t')
   time = progress[['Time']].to_numpy()
   times.append(np.squeeze(time[19]))
for i in np.linspace(19, 31, num=4,dtype=int):
   progress = pd.read_csv('/content/drive/MyDrive/Project/experiments2/PPO-'+str(i)+'/progress.txt',sep='\t')
   time = progress[['Time']].to_numpy()
   times.append(np.squeeze(time[19]))
Times2['Times']=times
print('Table 2. Total training time of different configurations for 200000 total environment interactions (20 epochs, 1500 steps per epoch)')
table3 = pd.pivot_table(Times2,values="Times",index=["Act_Function","Hidden_Layers"],columns=["Algo"])
table3

Table 2. Total training time of different configurations for 200000 total environment interactions (20 epochs, 1500 steps per epoch)


Algo                              DDPG        PPO
Act_Function Hidden_Layers                       
ReLu         [100, 50, 25]  268.201522  41.952555
             [64, 64]       200.179358  34.515318
Tanh         [100, 50, 25]  253.684878  43.449210
             [64, 64]       207.082204  37.339403

From this table we can infer two things, first, DDPG in general takes more time for being trained; second, both algorithms shows to have good times for Hidden Layers [64,64] and Activation Function ReLu. Let us visualize the training Reward throuh time of this two specific configurations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()
avgRewardTrainingDDPG = pd.read_csv('/content/drive/MyDrive/Project/experiments2/DDPG-1/progress.txt',sep='\t')   #1
avgRewardTrainingPPO = pd.read_csv('/content/drive/MyDrive/Project/experiments2/PPO-17/progress.txt',sep='\t')    # 17

xDDPG =np.squeeze(np.array ( avgRewardTrainingDDPG[['TotalEnvInteracts']] ))
yDDPG = np.squeeze(np.array (avgRewardTrainingDDPG[['AverageEpRet']]))
xPPO = np.squeeze(np.array ( avgRewardTrainingPPO[['TotalEnvInteracts']]))
yPPO =np.squeeze(np.array ( avgRewardTrainingPPO[['AverageEpRet']]))


# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter( y=yDDPG, mode='lines', name='DDPG'))
fig.add_trace(go.Scatter(y=yPPO, mode='lines', name='PPO'))
fig.update_layout(title='Average Reward 30000 total steps',xaxis_title='Epochs (Timesteps x1500)',yaxis_title='Average Reward per epoch')
fig.show()

#sns.set_theme(style="darkgrid")
avgRewardTrainingDDPG = pd.read_csv('/content/drive/MyDrive/Project/experiments2/DDPG-15/progress.txt',sep='\t')   #1
avgRewardTrainingPPO = pd.read_csv('/content/drive/MyDrive/Project/experiments2/PPO-31/progress.txt',sep='\t')    # 17


xDDPG =np.squeeze(np.array ( avgRewardTrainingDDPG[['TotalEnvInteracts']] ))
yDDPG = np.squeeze(np.array (avgRewardTrainingDDPG[['AverageEpRet']]))
xPPO = np.squeeze(np.array ( avgRewardTrainingPPO[['TotalEnvInteracts']]))
yPPO =np.squeeze(np.array ( avgRewardTrainingPPO[['AverageEpRet']]))

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter( y=yDDPG,mode='lines',name='DDPG'))
fig.add_trace(go.Scatter(y=yPPO,mode='lines',name='PPO'))
fig.update_layout(title='Average Reward 200000 total steps',xaxis_title='Epochs (Timesteps x2000)',yaxis_title='Average Reward per epoch')
fig.show()

# **Experiments with DDPG, PPO and PID**
Now that DDPG and PPO are already trained, let us run some experiments to check the performance of these algirhtms and also the common control technique PID. The next experiments will run the three algorithms starting from four different angle positions: 0.0001, 0.015, 0.02, 0.029. The results will be compared by using two specific metrics: **settling time** and **energy**. The first one is defined as the time elapsed from the starting moment in which the controller is applied to the time at which the system has entered and remained within a specified error band (this error is measured as the difference between the current angle position and the angle zero, i.e. up-right position). The second metric is assumed to be proportonial to the absolute value of the actions taken by the controller. Let us recall that for both DDPG and PPO the configuration chosen for these experiments is ReLU [64,64]

In [ ]:
from numpy.core.numerictypes import obj2sctype
import scipy.stats as st
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from spinup import ppo_pytorch as ppo
from spinup import ddpg_pytorch as ddpg
import torch
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

startPoints = np.array([0.000001,0.015,0.025,0.29])
iter=400
cons = 0.1
modelPath = '/content/drive/MyDrive/Project/experiments2/DDPG-15/pyt_save/model.pt'    #15 #1
acDDPG = torch.load(modelPath)  
modelPath ='/content/drive/MyDrive/Project/experiments2/PPO-31/pyt_save/model.pt'   # 31  # 17
acPPO = torch.load(modelPath) 
AverageReward = []
tableGen = []
perturbationConstant = 0.00001
Kp= 3.4210526315789473   
Ki= 0.035 
Kd = 0.027222222222222224 
energy = 0

for i in range(len(startPoints)):
  #env = gym.make("InvertedPendulum-v2")

  ####### DDPG #########
  env.reset()
  energy = []
  rewardsDDPG =[]
  for m in range (10):  # Ten episodes
    rewards =0
    actionsDDPG= []
    anglesDDPG = []  
    old_state = env.sim.get_state()
    new_state = mujoco_py.MjSimState(old_state.time, np.array([0 , startPoints[i]]), np.array([0 ,0]), old_state.act, old_state.udd_state)
    env.sim.set_state(new_state)
    obs =[new_state.qpos[0],new_state.qpos[1], new_state.qvel[0],new_state.qvel[1]]
    anglesDDPG.append(obs[1])
    done = False          
    for j in range(iter):      # 300 steps per episode
        action = acDDPG.act(torch.as_tensor(obs, dtype=torch.float32))
        if j==100:
           action = action +perturbationConstant* np.random.rand()
        obs, reward, done, info = env.step(action)
        rewards = rewards +reward
        anglesDDPG.append(obs[1])  
        actionsDDPG.append(action)
    EnergyDDPG = cons*sum(np.absolute(actionsDDPG))
    rewardsDDPG.append(rewards)     # Total reward per episode 
    energy.append(EnergyDDPG)       # total energy per episode
  meanEnergy=np.mean(energy)
  meanReward=np.mean(rewards)
  intervals =np.squeeze( st.t.interval(alpha=0.95, df=len(energy)-1, loc=np.mean(energy), scale=st.sem(energy)))
  st_dev = np.squeeze(np.std(energy))
  energy = np.squeeze(energy)
  tableGen.append((startPoints[i],meanReward,np.squeeze(rewardsPID),meanEnergy,energy,intervals,st_dev,'DDPG'))

  ######### PPO ####
  env.reset()
  energy=[]
  rewardsPPO =[]
  for m in range(10):
    rewards = 0
    anglesPPO = []
    actionsPPO= []  
    old_state = env.sim.get_state()
    new_state = mujoco_py.MjSimState(old_state.time, np.array([0 ,  startPoints[i]]), np.array([0 ,0]), old_state.act, old_state.udd_state)
    env.sim.set_state(new_state)
    obs =[new_state.qpos[0],new_state.qpos[1], new_state.qvel[0],new_state.qvel[1]]
    anglesPPO.append(obs[1])
    done = False      
    for j in range(iter): 
        action = acPPO.act(torch.as_tensor(obs, dtype=torch.float32))
        if j==100:
           action = action +perturbationConstant* np.random.rand()
        obs, reward, done, info = env.step(action)
        rewards = rewards +reward
        anglesPPO.append(obs[1])        
        actionsPPO.append(action)
    EnergyPPO = cons*sum(np.absolute(actionsPPO))
    rewardsPPO.append(rewards)    # Total reward per episode
    energy.append(EnergyPPO)      # Total energy per episode
  meanEnergy=np.mean(energy)
  meanReward=np.mean(rewards)
  intervals =np.squeeze( st.t.interval(alpha=0.95, df=len(energy)-1, loc=np.mean(energy), scale=st.sem(energy)))
  st_dev = np.squeeze(np.std(energy))
  energy = np.squeeze(energy)
  tableGen.append((startPoints[i],meanReward,np.squeeze(rewardsPID),meanEnergy,energy,intervals,st_dev,'PPO')) 

    ########## PID ########

  energy = []
  rewardsPID = []
  for m in range(10):
    env.reset()
    rewards=0
    anglesPID = []
    actionsPID = []
    old_state = env.sim.get_state()
    new_state = mujoco_py.MjSimState(old_state.time, np.array([0 ,  startPoints[i]]), np.array([0 ,0]), old_state.act, old_state.udd_state)
    env.sim.set_state(new_state)
    anglesPID.append(new_state.qpos[1])              
    goal = 0.0
    e = 0.0
    e1 = 0.0
    e2 = 0.0
    action = -0.012
    action_old = 0.0
    done = False    
  
    for j in range(iter):
          obs, reward, done, info = env.step(-action)
          anglesPID.append(obs[1])
          
          actionsPID.append(action)
          action_old = action
          e2 = e1
          e1 = e
          e = goal - obs[1] 
          action = action_old + Kp * (e-e1) + Ki * e + Kd * ((e-e1)-(e1-e2))
          
          if j==100:
           action = action +perturbationConstant* np.random.rand()
          if done ==False:
            rewards = rewards + reward
    EnergyPID = cons*sum(np.absolute(actionsPID))
    energy.append(EnergyPID)    # Total energy per episode
    rewardsPID.append(rewards)    # Total reward per episode
  meanEnergy=np.mean(energy)
  meanReward=np.mean(rewards)
  intervals =np.squeeze( st.t.interval(alpha=0.95, df=len(energy)-1, loc=np.mean(energy), scale=st.sem(energy)))
  st_dev = np.squeeze(np.std(energy))
  energy = np.squeeze(energy)
  tableGen.append((startPoints[i],meanReward,np.squeeze(rewardsPID),meanEnergy,energy,intervals,st_dev,'PID'))

  fig = go.Figure()
  x = np.linspace(1,iter,iter)
  fig.add_trace(go.Scatter(x=x, y=np.squeeze(np.array(anglesDDPG)), mode='lines',name='DDPG'))
  fig.add_trace(go.Scatter(x=x,y=np.squeeze(np.array(anglesPPO)),mode='lines',name='PPO'))
  fig.add_trace(go.Scatter(x=x,y=np.squeeze(np.array(anglesPID)),mode='lines',name='PID'))
  fig.update_layout(title='Controller performance with starting point '+str(startPoints[i]),xaxis_title='Environment steps',yaxis_title='Error (radians)')
  fig.show()
df = pd.DataFrame(tableGen,columns =['Starting_point', 'Mean_Reward','Total_reward','Ave_Total_energy','Energy','intervals','Stand_dev', 'Algo'])
df.to_csv("/content/drive/MyDrive/Project//TableGen200000_6.csv",sep='\t')  

In [ ]:
import pandas as pd
results = pd.read_csv('/content/drive/MyDrive/Project/TableGen30000_6.csv',sep='\t')

table4 = pd.pivot_table(results,values=["Ave_Total_energy","Stand_dev",'Mean_Reward'],index="Starting_point",columns="Algo")
print('Table 4.Total energy expended by every controller 30000, Relu [64,64]')
table4


Table 4.Total energy expended by every controller 30000, Relu [64,64]


Ave_Total_energy                        Mean_Reward         \
Algo                       DDPG         PID        PPO        DDPG    PID   
Starting_point                                                              
0.000001              33.297184    0.183560  23.344128       400.0  400.0   
0.015000              34.006325    0.590489  23.689526       400.0  400.0   
0.025000              34.809368    1.444071  24.019897       400.0  396.0   
0.290000              32.353493  489.070794  23.758457       400.0   38.0   

                      Stand_dev                      
Algo              PPO      DDPG       PID       PPO  
Starting_point                                       
0.000001        400.0  0.364908  0.000006  0.772474  
0.015000        400.0  0.627066  0.000008  0.935974  
0.025000        400.0  0.651144  0.008471  0.615026  
0.290000        400.0  0.419433  0.000040  0.689986

In [ ]:
import pandas as pd
results = pd.read_csv("/content/drive/MyDrive/Project/TableGen200000_6.csv",sep='\t')
table5 = pd.pivot_table(results,values=["Ave_Total_energy","Stand_dev","Mean_Reward"],index="Starting_point",columns="Algo")
print('Table 4.Total energy expended by every controller 200000, Tanh [100,50,25]')
table5

Table 4.Total energy expended by every controller 200000, Tanh [100,50,25]


Ave_Total_energy                        Mean_Reward         \
Algo                       DDPG         PID        PPO        DDPG    PID   
Starting_point                                                              
0.000001              74.936638    0.183561  16.403093       400.0  400.0   
0.015000              69.627777    0.590487  16.238972       400.0  400.0   
0.025000              72.665237    1.444298  16.134586       400.0  396.0   
0.290000              69.469017  489.070707  16.487576       400.0   38.0   

                      Stand_dev                      
Algo              PPO      DDPG       PID       PPO  
Starting_point                                       
0.000001        400.0  0.896009  0.000012  0.701142  
0.015000        400.0  1.326447  0.000009  0.634864  
0.025000        400.0  0.864852  0.008229  0.793379  
0.290000        400.0  0.600341  0.000083  0.454569

# Visualize DDPG results

In [ ]:
def save_video_of_model( env_name, filename,path):  
  import torch
  import skvideo.io
  from pyvirtualdisplay import Display
  display = Display(visible=0, size=(40, 30))
  display.start()

  actions = []
  angles  = [] 
  env = gym.make(env_name)
  obs = env.reset()
  shape = env.render(mode='rgb_array').shape[0:2]

  out = skvideo.io.FFmpegWriter(filename)
  ac = torch.load(path)
  done = False
  while not done: 
      frame = env.render(mode='rgb_array')
      out.writeFrame(frame)
      action = ac.act(torch.as_tensor(obs, dtype=torch.float32))
      actions.append(action)
      obs, reward, done, info = env.step(action)
      angles.append(obs[2])
      
  out.close()
  print( "Successfully saved into {}!".format(filename))

save_video_of_model( "InvertedPendulum-v2",filename='agentDDPG.mp4',path='/content/drive/MyDrive/Project/experiments2/DDPG-1/pyt_save/model.pt')

Successfully saved into agentDDPG.mp4!


In [ ]:
from IPython.display import HTML
import io, base64
video = io.open('./agentDDPG.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
<video controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
</video>'''.format(encoded.decode('ascii')))